# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7.1

In [141]:
import numpy as np
import pandas as pd
import xlwings as xw
import csv

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [78]:
reviews = pd.read_csv('data/reviews_sample.csv',
                     index_col=0,
                     parse_dates=['date'])
reviews

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...
1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."
453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...


In [79]:
recipes = pd.read_csv('data/recipes_sample.csv', parse_dates=['submitted'])[['name', 'id', 'minutes', 'contributor_id', 'submitted', 'description', 'n_ingredients']]
recipes

,name,id,minutes,contributor_id,submitted,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,227978,2012-08-29,this is a delicious soup that i originally fou...,NaN


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [80]:
get_5 = lambda x: x.sample(x.shape[0]//20) # Функция возвращает 5% случайных строк из переданного df 

In [81]:
sample_recipes = get_5(recipes)
sample_reviews = get_5(reviews)

In [82]:
book = xw.Book('files/book1.xlsx') # Считываем таблицу
book.sheets[0].name = 'recipes' # Переименовываем таблицу
xl_recipes = book.sheets['recipes'] # Записываем таблицу recipes в переменную
xl_recipes.range('A1').value = sample_recipes # Заполняем её

In [83]:
xl_reviews = book.sheets.add(name='reviews') # Добавляем таблицу
xl_reviews.range('A1').value = sample_reviews 

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [90]:
xl_recipes.range('I2').options(transpose=True, ndim=1).value = [minutes*60 for minutes in sample_recipes['minutes']]
xl_recipes.range('I1').value = 'seconds_assign' # Прописываем название столбцу

In [85]:
rng = xw.Range((1, 1), (1501, 10))
for i in range(len(rng.columns)):
    rng.columns[i].autofit()

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [86]:
formula = '=D2*60' # Создаём формулу для заполнения столбца
xl_recipes.range('J1').value = 'seconds_formula' # Прописываем название столбцу
fml = xw.Range('J2').formula = formula # Назначаем 
xl_recipes.range('J2:J1501').formula = fml

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [96]:
xl_recipes.range('I1:J1').font.bold = True
xl_recipes.range('I1:J1').api.HorizontalAlignment = -4108

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [114]:
mins = xl_recipes.range('D2:d1501')
for i in mins:
    if i.value > 10:
        i.color = (255, 50, 50)
    elif i.value < 5:
        i.color = (50, 255, 50)
    else:
        i.color = (250, 250, 100)
#     print(i.value)
# mins.color = None

KeyboardInterrupt: 

In [113]:
mins = xl_recipes.range('D2:D1501')
mins.color = None

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [123]:
formula = '=COUNTIF(reviews!C:C, C2)'
xl_recipes.range('K2:K1501').formula = formula

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [138]:
import xlwings as xw

def validate():
    # Подключение к Excel
    wb = xw.Book('Book1.xlsx')
    xl_reviews = wb.sheets['reviews']
    xl_recipes = wb.sheets['recipes']
    # Получение диапазона данных из листа "Отзывы"
    reviews_data = xl_reviews.range('A2:F1501').value
    # Перебор каждой строки данных
    for row in reviews_data:
        rating = int(row[4])
        recipe_id = row[2]
        if 0 <= rating <= 5 and recipe_id in xl_recipes:
            xl_reviews.range(f'A{reviews_data.index(row)+2}:C{reviews_data.index(row)+2}').color = (50, 250, 50)

validate()

com_error: (-2147352567, 'Ошибка.', (0, None, None, None, 0, -2147352566), None)

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

In [160]:
with open('data/recipes_model.csv', encoding="utf-8") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='	')
    for line_count, row in enumerate(csv_reader, 0):
        print(row)
    xl = xw.book = xw.Book('files/recipes_model.xlsx') # Считываем таблицу


{'recipe': 'recipe', 'name': 'id', 'varchar(100)': 'serial', 'Y': 'N', 'Рецепт': 'Рецепт', 'Название рецепта': 'ID рецепта', '': ''}
{'recipe': 'recipe', 'name': 'minutes', 'varchar(100)': 'int', 'Y': 'Y', 'Рецепт': 'Рецепт', 'Название рецепта': 'Длительность выполнения в минутах', '': ''}
{'recipe': 'recipe', 'name': 'contributor_id', 'varchar(100)': 'int', 'Y': 'Y', 'Рецепт': 'Рецепт', 'Название рецепта': 'ID автора рецепта', '': 'id'}
{'recipe': 'recipe', 'name': 'submitted', 'varchar(100)': 'timestamp', 'Y': 'Y', 'Рецепт': 'Рецепт', 'Название рецепта': 'Дата добавления', '': ''}
{'recipe': 'recipe', 'name': 'n_steps', 'varchar(100)': 'int', 'Y': 'N', 'Рецепт': 'Рецепт', 'Название рецепта': 'Кол-во шагов рецепта', '': ''}
{'recipe': 'recipe', 'name': 'description', 'varchar(100)': 'text', 'Y': 'N', 'Рецепт': 'Рецепт', 'Название рецепта': 'Описание рецепта', '': ''}
{'recipe': 'recipe', 'name': 'n_ingredients', 'varchar(100)': 'int', 'Y': 'N', 'Рецепт': 'Рецепт', 'Название рецепта': 

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 